In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Folder containing the images
image_folder = '/content/drive/My Drive/viscoelastic/3% HPMC/measurement viscoelastic_001/'

# List to store the measurements
measurements = []

# Function to process each image and measure the droplet diameter
def process_image(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Step 1: Noise removal (Gaussian blur)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Step 2: Sharpening
    sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened = cv2.filter2D(blurred, -1, sharpen_kernel)

    # Step 3: Edge detection (Canny edge detection)
    edges = cv2.Canny(sharpened, threshold1=50, threshold2=150)

    # Step 4: Thresholding
    _, thresh = cv2.threshold(edges, 127, 255, cv2.THRESH_BINARY)

    # Step 5: Fill the holes
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_image = np.zeros_like(thresh)
    cv2.drawContours(filled_image, contours, -1, 255, thickness=cv2.FILLED)

    # Step 7: Outline the droplet
    outline = cv2.drawContours(np.zeros_like(filled_image), contours, -1, 255, thickness=2)

    # Step 8: Calculate the diameters along the x and y axes
    # Assuming the largest contour corresponds to the droplet
    if contours:
        cnt = min(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)

        diameter_x = w  # Diameter parallel to the x-axis
        diameter_y = h  # Diameter perpendicular to the x-axis
        return diameter_x, diameter_y
    else:
        return None, None

# Process all images in the folder
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    if image_path.endswith('.tif'):  # Filtering for .tif images
        diameter_x, diameter_y = process_image(image_path)
        if diameter_x and diameter_y:
            measurements.append({
                'image': image_name,
                'diameter_x': diameter_x,
                'diameter_y': diameter_y
            })
            print(f'Processed {image_name} - Diameter x: {diameter_x}, Diameter y: {diameter_y}')
        else:
            print(f'No droplet found in {image_name}')

# Save the results to a CSV file
import pandas as pd
df = pd.DataFrame(measurements)
df.to_csv('droplet_measurements.csv', index=False)

print('All images processed. Results saved to droplet_measurements.csv')
